# 02 — Agent Architectures in ADK  
Multi-Agent Systems & Workflow Patterns  
A clean local adaptation of Kaggle’s Day-1b notebook (no Kaggle secrets, no Google Search tool).

In this notebook we:

- Configure ADK for local use
- Build a multi-agent “research + summary” system
- Implement **Sequential**, **Parallel**, and **Loop** workflow agents
- Use `output_key` and shared state to let agents collaborate


## Setup & Imports

In [1]:
# --- Load environment variables ---
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not set — ensure it is in your .env file")

print("API key loaded.")


API key loaded.


## ADK + Generative Imports

In [2]:
# --- Core ADK & Gemini components ---
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool

from google.genai import types  # generative types: Content, Part, HttpRetryOptions

print("ADK imports loaded successfully.")


ADK imports loaded successfully.


## Retry Config (

In [3]:
# --- Retry configuration (matches Kaggle logic, but used locally) ---
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("Retry config ready.")


Retry config ready.


## Session constants & helper runner

In [4]:
# --- Shared constants for this notebook ---
APP_NAME = "google_ai_agents_course"
USER_ID = "local_user_001"

print(f"APP_NAME={APP_NAME}, USER_ID={USER_ID}")


# --- Helper coroutine to run any root agent once ---
async def run_root_agent(root_agent, question: str, session_id: str):
    """
    Run a single turn with a given root agent using InMemoryRunner.

    Improvements vs. original:
    - Filters out tool-call artifacts (e.g., <ctrl42>call)
    - Captures session state updates from LoopAgent
    - Returns the final story from state if available
    """
    runner = InMemoryRunner(
        agent=root_agent,
        app_name=APP_NAME,
    )

    # Ensure session exists
    await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session_id,
    )

    # Build user message
    user_message = types.Content(
        role="user",
        parts=[types.Part(text=question)]
    )

    final_text = None
    final_state = None  # capture story from LoopAgent

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=user_message,
    ):
        # Capture state updates (LoopAgent writes refined stories here)
        if hasattr(event, "session_state") and event.session_state:
            final_state = event.session_state

        # Capture *only real text parts*, ignore tool/function calls
        if event.is_final_response() and event.content:
            parts_list = event.content.parts or []  # <-- SAFETY FIX

            parts = [
                p.text for p in parts_list
                if getattr(p, "text", None)
            ]
            if parts:
                final_text = "".join(parts)

    # If LoopAgent produced a refined story, return that
    if final_state and "current_story" in final_state:
        return final_state["current_story"]

    return final_text or final_state


APP_NAME=google_ai_agents_course, USER_ID=local_user_001


## Multi-Agent “Research + Summary” via AgentToo

## 2 — Multi-Agent System: Research & Summarization

We recreate the Kaggle “ResearchAgent + SummarizerAgent + Coordinator” pattern.

Differences vs Kaggle:

- We **do not** use `google_search` (no live web).
- `ResearchAgent` still behaves like a research agent, but it uses the model’s own knowledge.
- `AgentTool(research_agent)` and `AgentTool(summarizer_agent)` are preserved so the root agent orchestrates the workflow.


## Define Research & Summarizer agents

In [5]:
# ResearchAgent: behaves like a research specialist (no explicit tools here).
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "You are a specialized research agent. "
        "Your only job is to gather 2–3 key points of relevant information "
        "about the given topic and present them clearly with brief citations "
        "or source-style hints where possible. "
        "If you are unsure or lack current data, say so explicitly."
    ),
    tools=[],  # no google_search in the local environment
    output_key="research_findings",
)

print("research_agent created.")


research_agent created.


In [6]:
# SummarizerAgent: turns research findings into a concise bulleted summary.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Read the provided research findings: {research_findings}\n\n"
        "Create a concise summary as a bulleted list with 3–5 key points. "
        "Focus on clarity and high-level takeaways."
    ),
    output_key="final_summary",
)

print("summarizer_agent created.")


summarizer_agent created.


## Define Coordinator with AgentTool

In [7]:
# Root coordinator: orchestrates ResearchAgent + SummarizerAgent via AgentTool.
root_research_coordinator = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "You are a research coordinator.\n"
        "Your goal is to answer the user's query by orchestrating this workflow:\n"
        "1. Call `ResearchAgent` to gather findings on the user's topic.\n"
        "2. After the research concludes, call `SummarizerAgent` to produce a concise summary.\n"
        "3. Present the final summary clearly to the user as your response.\n"
        "Do not invent tools; rely only on the available agent tools."
    ),
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent),
    ],
)

print("root_research_coordinator created.")


root_research_coordinator created.


## Test run

In [8]:
question = "What are current applications and challenges of quantum computing for AI?"

answer = await run_root_agent(
    root_agent=root_research_coordinator,
    question=question,
    session_id="day01b_multi_agent_001",
)

print(answer)


Quantum computing shows promise for accelerating AI tasks like machine learning and optimization, potentially revolutionizing fields such as drug discovery and finance. However, the technology is still in its early stages, facing significant challenges related to hardware limitations (noise, errors, scalability) and the need for specialized expertise. Currently, practical, large-scale quantum computers capable of outperforming classical machines for most AI tasks are not yet available.


## Sequential Workflows — Blog Pipeline

## SequentialAgent: Blog Creation Pipeline

We now build a **deterministic assembly line**:

1. `OutlineAgent` – creates a blog outline  
2. `WriterAgent` – writes a draft using `{blog_outline}`  
3. `EditorAgent` – polishes the `{blog_draft}` into `final_blog`

All plumbing is driven by `output_key` and `{placeholder}` substitution in instructions, coordinated by `SequentialAgent`.


## Define outline, writer, and editor agents

In [9]:
# OutlineAgent: creates the skeleton of the blog post.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Create a blog outline for the given topic with:\n"
        "1. A catchy headline\n"
        "2. An introduction hook\n"
        "3. 3–5 main sections with 2–3 bullet points each\n"
        "4. A concluding thought\n"
        "Return only the outline, clearly structured."
    ),
    output_key="blog_outline",
)

print("outline_agent created.")


outline_agent created.


In [10]:
# WriterAgent: writes the blog following the outline exactly.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Following this outline strictly:\n"
        "{blog_outline}\n\n"
        "Write a brief, 200–300 word blog post with an engaging and "
        "informative tone. Do not include the outline itself; only the article."
    ),
    output_key="blog_draft",
)

print("writer_agent created.")


writer_agent created.


In [11]:
# EditorAgent: polishes the draft.
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Edit this draft for clarity, flow, and grammar. Improve sentence "
        "structure but keep the original meaning and tone:\n\n"
        "{blog_draft}\n\n"
        "Return only the revised blog post."
    ),
    output_key="final_blog",
)

print("editor_agent created.")


editor_agent created.


## Wrap them in a SequentialAgent

In [12]:
root_blog_pipeline = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("Sequential BlogPipeline created.")


Sequential BlogPipeline created.


## Test run

In [13]:
topic = "The benefits of multi-agent systems for software developers"

blog_result = await run_root_agent(
    root_agent=root_blog_pipeline,
    question=f"Write a blog post about: {topic}",
    session_id="day01b_blog_pipeline_001",
)

print(blog_result)


### Supercharge Your Software: How Multi-Agent Systems Are Revolutionizing Development

Are you struggling with monolithic codebases and complex, difficult-to-manage systems? Picture a software architecture where independent, intelligent units collaborate seamlessly to achieve intricate goals. This is the power of multi-agent systems (MAS), and they are rapidly becoming essential tools for modern software developers, moving beyond the realm of AI research.

A key benefit of MAS lies in their contribution to enhanced **modularity and maintainability**. Instead of wrestling with a massive codebase, you can decompose complex problems into smaller, more manageable agent modules. This approach significantly simplifies the process of updating, replacing, or debugging individual agents without disrupting the entire system. Moreover, this design promotes code reusability and enables development teams to specialize in particular agent functionalities, thereby boosting overall efficiency.

MAS a

## Parallel Workflows — Multi-Topic Research

## 4 — ParallelAgent: Multi-Topic Research

Now we want **independent research tasks** to run concurrently:

- `TechResearcher`
- `HealthResearcher`
- `FinanceResearcher`

Their outputs (`tech_research`, `health_research`, `finance_research`)
are then combined by `AggregatorAgent` into a single executive summary.

We nest a `ParallelAgent` inside a `SequentialAgent`:

1. Parallel team runs all researchers concurrently  
2. AggregatorAgent runs after all research outputs are in state


## Define the three “researcher” agents
All without `google_search`, but instructed as domain specialists.

In [14]:
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Research the latest AI/ML trends. Include ~3 key developments, "
        "the main companies involved, and their potential impact. "
        "Keep the report very concise (≈100 words). "
        "If you are uncertain about dates, be explicit."
    ),
    tools=[],  # no external search
    output_key="tech_research",
)

print("tech_researcher created.")


tech_researcher created.


In [15]:
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Research recent medical or biotech breakthroughs. Include ~3 "
        "significant advances, their practical applications, and rough "
        "timelines. Keep the report concise (≈100 words)."
    ),
    tools=[],
    output_key="health_research",
)

print("health_researcher created.")


health_researcher created.


In [16]:
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Research current fintech trends. Include ~3 key trends, their "
        "market implications, and the future outlook. "
        "Keep the report concise (≈100 words)."
    ),
    tools=[],
    output_key="finance_research",
)

print("finance_researcher created.")


finance_researcher created.


## AggregatorAgent

In [17]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Combine these three research findings into a single executive summary:\n\n"
        "Technology Trends:\n{tech_research}\n\n"
        "Health Breakthroughs:\n{health_research}\n\n"
        "Finance Innovations:\n{finance_research}\n\n"
        "Highlight common themes, surprising connections, and key takeaways in "
        "≈200 words."
    ),
    output_key="executive_summary",
)

print("aggregator_agent created.")


aggregator_agent created.


## Parallel + Sequential composition

In [18]:
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_parallel_system = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("ParallelResearchTeam + ResearchSystem created.")


ParallelResearchTeam + ResearchSystem created.


## Test run

In [19]:
exec_briefing = await run_root_agent(
    root_agent=root_parallel_system,
    question="Run the daily executive briefing on Tech, Health, and Finance.",
    session_id="day01b_parallel_research_001",
)

print(exec_briefing)


## Executive Summary: Convergence of AI, Health, and Finance Innovations

This briefing highlights the pervasive and accelerating impact of Artificial Intelligence (AI) across technology, health, and finance. A key common thread is the application of advanced AI and Machine Learning (ML) to drive significant breakthroughs and efficiencies.

In **health**, AI is revolutionizing drug discovery, drastically shortening development timelines for treatments of complex diseases like cancer and Alzheimer's. Concurrently, CRISPR gene therapies are nearing wider clinical application, offering potential cures for inherited disorders within the next 3-5 years. The success of mRNA technology is also being leveraged for new vaccines against infectious diseases and personalized cancer therapies.

**Technology** trends show rapid advancements in Generative AI for content creation and AI's critical role in scientific discovery, mirroring its use in health. The growing adoption of Edge AI emphasizes enh

## Loop Workflows — Story Refinement

## 5 — LoopAgent: Writer + Critic Refinement Cycle

We now create an **iterative refinement loop**:

1. `InitialWriterAgent` produces the first story draft (`current_story`)
2. `CriticAgent` evaluates the story and either:
   - says `"APPROVED"` or
   - returns feedback in `critique`
3. `RefinerAgent`:
   - calls `exit_loop()` via `FunctionTool` if critique == `"APPROVED"`, or
   - rewrites the story and updates `current_story`

`LoopAgent` coordinates repeated `Critic → Refiner` cycles, bounded by `max_iterations`.


## Initial writer & critic

In [20]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "Based on the user's prompt, write the first draft of a short story "
        "(around 100–150 words). Output only the story text, with no "
        "introduction or explanation."
    ),
    output_key="current_story",
)

print("initial_writer_agent created.")


initial_writer_agent created.


In [21]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "You are a constructive story critic. Review the story below.\n\n"
        "Story: {current_story}\n\n"
        "Evaluate the story's plot, characters, and pacing.\n"
        "- If the story is well-written and complete, you MUST respond with "
        'the exact phrase: "APPROVED".\n'
        "- Otherwise, provide 2–3 specific, actionable suggestions "
        "for improvement."
    ),
    output_key="critique",
)

print("critic_agent created.")


critic_agent created.


## Exit function + FunctionTool & RefinerAgent

In [22]:
# Function used as explicit loop-termination signal
def exit_loop():
    """
    Called ONLY when the critique is 'APPROVED', indicating the story
    is finished and no more changes are needed.
    """
    return {
        "status": "approved",
        "message": "Story approved. Exiting refinement loop.",
    }


print("exit_loop() function defined.")


exit_loop() function defined.


In [23]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=(
        "You are a story refiner. You have a story draft and a critique.\n\n"
        "Story Draft:\n{current_story}\n\n"
        "Critique:\n{critique}\n\n"
        "Your task:\n"
        '- IF the critique is EXACTLY "APPROVED", you MUST call the '
        "`exit_loop` function and do nothing else.\n"
        "- OTHERWISE, rewrite the story draft to fully incorporate the "
        "feedback. Output only the revised story text."
    ),
    output_key="current_story",  # overwrite with refined story
    tools=[FunctionTool(exit_loop)],
)

print("refiner_agent created.")


refiner_agent created.


## LoopAgent + outer Sequential pipeline

In [24]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # safeguard against infinite loops
)

root_story_pipeline = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("StoryPipeline with LoopAgent created.")


StoryPipeline with LoopAgent created.


## Test run

In [25]:
story_prompt = (
    "Write a short story about a lighthouse keeper who discovers a "
    "mysterious, glowing map."
)

refined_story = await run_root_agent(
    root_agent=root_story_pipeline,
    question=story_prompt,
    session_id="day01b_story_loop_001",
)

print(refined_story)


APPROVED


## Summary

## 6 — Summary: Choosing the Right Pattern

Quick mental model:

| Pattern        | When to Use                               | Example                                  |
|---------------|-------------------------------------------|------------------------------------------|
| LLM-based root + AgentTool | Dynamic orchestration by an LLM | Research + Summarize system              |
| SequentialAgent | Order matters, strict pipeline         | Outline → Write → Edit blog pipeline     |
| ParallelAgent   | Independent tasks, speed matters       | Multi-topic research (Tech/Health/Fin)   |
| LoopAgent       | Iterative refinement and stopping logic | Writer + Critic story refinement loop    |

Your local notebook now mirrors the **architecture** of the Kaggle Day-1b notebook, but:
- Uses your `.env` API key
- Avoids Kaggle secrets and `google_search`
- Keeps the ADK patterns you’ll reuse in real projects
